# Synthetic parameters

Получение параметров модели методом подгонки синтетического $ P(t) $ под измеренное.

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import sys
sys.path.insert(0, './python')

In [ ]:
from windkessel import model, utils

In [ ]:
import re
from os import listdir

filenames = listdir("./data")
IDs = [int(filename[0:-4]) for filename in filenames if re.match(r".\d*\.csv$", filename)]

## Данные о пациентах

In [ ]:
def prepare_data(data_frame):
    data_frame = data_frame[data_frame.index.notnull()]
    data_frame.index = data_frame.index.astype('uint64')
    data_frame = data_frame.drop(columns=["ФИО"])
    return data_frame

In [ ]:
data_patients = prepare_data(pd.read_excel(f'data/data_patients.xlsx', index_col="ID"))
data_PWV = prepare_data(pd.read_excel(f'data/data_PWV.xlsx', index_col="ID"))

In [ ]:
data_patients = data_patients[data_patients.index.isin(IDs)]
data_patients = data_patients.join(data_PWV["PWV_ba"])

In [ ]:
def get_params(patient_id):
    print(patient_id)
    
    # Создание модели.
    wk_model = model.WindkesselModel()
    
    # Получение данных.
    tp = pd.read_csv(f'data/{patient_id}.csv', header = None, sep = ' ')
    
    # t в секундах, p в мм.рт.ст.
    tp = tp.rename({0 :'t', 1: 'p'}, axis = 1)
    tp = tp.sort_values(by=['t'])
    
    T = np.array(tp['t'])
    P = np.array(tp['p'])
    
    # Фильтрация.
    n_points = 200
    interp1d_params = {}
    savgol_filter_params = {"window_length": 9, "polyorder": 3, "mode": "nearest"}

    T, P = utils.nonuniform_savgol_filter(T, P, n_points=n_points,
            interp1d_params=interp1d_params, savgol_filter_params=savgol_filter_params)

    # Производная.
    savgol_filter_params["deriv"] = 1
    T, der_P = utils.nonuniform_savgol_filter(T, P, n_points=n_points,
            interp1d_params=interp1d_params, savgol_filter_params=savgol_filter_params)
    
    wk_model.set_P(T, P, der_P)
    
    # Подсчёт C методом подгонки синтетических данных.
    result = wk_model.get_synthetic_artery_params()
    T_max, T_s, T_d, Q_max, R_f, R, Z_0, C, P_out = result.x
    
    return [T_max, T_s, T_d, Q_max, R_f, R, Z_0, C, P_out]

In [ ]:
windkessel_params = pd.DataFrame(index = data_patients.index)

In [ ]:
params = np.array([get_params(patient_id) for patient_id in data_patients.index])

windkessel_params["T_max"] = params[:,0]
windkessel_params["T_s"]   = params[:,1]
windkessel_params["T_d"]   = params[:,2]
windkessel_params["Q_max"] = params[:,3]
windkessel_params["R_f"]   = params[:,4]
windkessel_params["R"]     = params[:,5]
windkessel_params["Z_0"]   = params[:,6]
windkessel_params["C"]     = params[:,7]
windkessel_params["P_out"] = params[:,8]

In [ ]:
windkessel_params.to_csv("./data/windkessel_params.csv", sep=' ')